In [1]:
%run ".//impport_packages.ipynb"    #import all necessary packages - numpy, pandas etc
%run ".//simulation_class.ipynb"    #import all necessary packages - numpy, pandas etc
%run ".//create_world.ipynb"

### Reformulation

- Define the **maternal deviation**:


##### $\delta_t$= Phenotype of the mother - Genetic contribution of mother
##### $\delta_t$ = $P_{t-1} - G_{t-1}$

so that the phenotype is the sum of the genetic component of the current generation and maternal deviation from its phenotype:

$$
P = G_t + \omega * \delta_t + \varepsilon_{d}
$$

- We also separate mutation-related noise $\varepsilon_{m}$ from parental phenotype inheritance noise $\varepsilon_{epi}$.  
- We define a evolvable 'weight' term $\omega$: determines how much importance to give each term of the equation
- We also add developmental noise: purely environmental noise in teh form of  $\varepsilon_{d}$


The full new formulation becomes:

$$
Z' = (G_t + \varepsilon_{m}) + (\omega + \varepsilon_{epi})*(\delta_t) +  \varepsilon_{d}
$$

---

In [5]:
rho_cases = 20
m_cases= 21
maxgen= 500
popsize = 1000

full_env_values = create_world(rho_cases, m_cases, maxgen)   # create world (input: generations, no. rhos and no. ms) 
print(full_env_values.shape)
clean_data_env = full_env_values[~np.isnan(full_env_values).any(axis=1)]
clean_data_env.shape                                         

(420, 504)


(283, 504)

In [6]:
len(np.unique([full_env_values[:,0]]))
len(np.unique([clean_data_env[:,0]]))

20

In [7]:
rho_m_alpha_beta= clean_data_env[:,0:4]
env_states= clean_data_env[:,4:]


In [8]:
nof_scenarios = np.shape(clean_data_env)[0]
print('nof_scenarios:', nof_scenarios)

nof_scenarios: 283


In [9]:

# Mutation rates: [epi-memory mutation rate, geno mutation rate]
mutrate = [0.01, 0.01]

# Mutation sizes: [epi-memory mutation size, geno mutation size]
mutsize = [1, 1]
rho_m_alpha_beta
epsilon = 0.1 # phenotypic noise

# Number of dimensions (can be 1 or more)
nof_dimensions = 4


In [10]:

# 0-based indexing
memory = 0
neutral = 1

# Indices for sections
geno = list(range(2, 2 + nof_dimensions))
pheno = list(range(2 + nof_dimensions, 2 + 2 * nof_dimensions))
dev = list(range(2 + 2 * nof_dimensions, 2 + 3 * nof_dimensions))


In [11]:
# check
print("memory =", memory)
print("neutral =", neutral)
print("geno =", geno)
print("pheno =", pheno)
print("dev =", dev)


memory = 0
neutral = 1
geno = [2, 3, 4, 5]
pheno = [6, 7, 8, 9]
dev = [10, 11, 12, 13]


In [12]:
# Preallocate a 3D NumPy array filled with NaN values.
# Shape explanation:
#   axis 0 → nof_scenarios  (number of scenarios from create worlds)
#   axis 1 → popsize        (number of individuals)
#   axis 2 → dev[-1]        (index of last developmental variable)
total_layers = (dev[-1]+1)
N = np.empty((nof_scenarios,popsize, total_layers)) 
# ↑ Same as N=nan(nof_scenarios,popsize,dev(end)); here the giant cube storing all info is initialised



In [13]:

N[:,:,memory:neutral]=-3 # initialize the  first layer + neutral trait to some arbitrary value of -3
N[:,:,geno]= np.zeros((nof_scenarios,popsize,nof_dimensions)) 
N[:,:,pheno]= N[:,:,geno] + epsilon* np.random.randn(nof_scenarios,popsize,nof_dimensions)
N[:,:,dev] = np.nan

In [14]:
sigma= 1

meanw = np.zeros((nof_scenarios,maxgen))
meanmemory_g = np.zeros((nof_scenarios,maxgen))
meanmemory_p= np.zeros((nof_scenarios,maxgen))
meanneutral_g = np.zeros((nof_scenarios,maxgen))
meanneutral_p = np.zeros((nof_scenarios,maxgen))

In [15]:
for t in range(0, (maxgen-1)):
    if t%10 == 0:
        print(t)
        
    
    for d in range(0, nof_dimensions):
           N[:,:,dev[d]] = N[:,:,pheno[d]] - env_states[:,t].reshape(-1,1)
           
    dev_combined = np.sqrt(np.sum(N[:,:,dev]**2, axis =2))     
    W = np.exp((-dev_combined**2)/ (2* sigma))
    
    
    meanw[:, t] = np.mean(W, axis= 1)    
    meanmemory_g[:,t] = np.mean(N[:,:,memory],axis= 1)
    meanmemory_p[:,t] = np.mean(1/(1 + np.exp(-N[:,:,memory])), axis= 1)
    meanneutral_g[:,t] = np.mean(N[:,:,neutral],axis= 1)
    meanneutral_p[:,t] = np.mean(1/(1 + np.exp(-N[:,:,neutral])), axis= 1)
    
    #current gen offspring (the cube gets stored here per gen)
    
    offspring = np.zeros((nof_scenarios,popsize,total_layers)) # empty 3D matrix with dimensions of the cube
    
    #sample offspring for each scenario weighted by fitness
    
    for scenario in range(0, nof_scenarios):
        parents_idx = np.random.choice(popsize, size=popsize, p = (W[scenario,]/np.sum(W[scenario,])) )#pick #popsize sized random numbers
        offspring[scenario,:,:] = N[scenario, parents_idx, :] #similar to how it works in matlab
    
    mutate_memories = np.random.uniform(low=0, high=1, size=(nof_scenarios, popsize)) < mutrate[0]
    mutate_neutral = np.random.uniform(low=0, high=1, size=(nof_scenarios, popsize)) < mutrate[0]
    mutate_geno = np.random.uniform(low=0, high=1, size=(nof_scenarios, popsize, nof_dimensions)) < mutrate[1]
    
    # adding a mutation of size mutsize[1], rate mutate_memories, to the epi-memory
    # the epignetic memory evolves; 
    offspring[:,:,memory] = (
                            offspring[:,:,memory] 
                            + mutate_memories * (mutsize[0] * np.random.uniform(low=0, high=1, size=(nof_scenarios, popsize)))
    )
    
    # adding a mutation of size mutsize[1] (epi-mutation size), rate mutate_memories, to the neutral phenotype
    # the neutral trait recieves random mutation but is not implemented in the calculation for fitness; so is evolving neutrally
    offspring[:,:,neutral] = (
                               offspring[:,:,neutral]  
                              + mutate_neutral * (mutsize[0] * np.random.uniform(low=0, high=1, size=(nof_scenarios, popsize)))
    )
    
    # adding a mutation of size mutsize[2] (genetic mutation size), rate mutate_memories, to the genotype
    
    offspring[:,:,geno] = (
                           offspring[:,:,geno] 
                          + mutate_geno * (mutsize[1] * np.random.uniform(low=0, high=1, size=(nof_scenarios, popsize, nof_dimensions )))
    )
    
    
    # form their phenotypes (we still know the parents)
    
    for i in range(0, nof_dimensions):
        offspring[:, :, pheno[i]] = (
        offspring[:, :, geno[i]] #genotype
        + epsilon * np.random.uniform(low=0, high=1, size=(nof_scenarios, popsize)) # adding random noise (developemental noise)
        + ((1 / (1 + np.exp(-offspring[:, :, memory])))  #phenotype version of the memory (y axis) 
        * (N[:, parents_idx, pheno[i]] - N[:, parents_idx, geno(i)])) # adding the deviation (for some reason?) -> read as weight (phenotypic component * deviation)
    )
    
    N=offspring
        

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490


In [49]:
results_dict = {
    'rho_m_alpha_beta' :rho_m_alpha_beta,
    'meanw': meanw,
    'meanmemory_g': meanmemory_g,
    'meanmemory_p': meanmemory_p,
    'meanneutral_g': meanneutral_g,
    'meanneutral_p': meanneutral_p
}

In [50]:
with open(f'results_dim{nof_dimensions}.pkl', 'wb') as f:
    pickle.dump(results_dict, f)